In [ ]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import re
from functools import partial
from io import StringIO

In [ ]:
import urllib.request


list_of_flags_page = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_national_flags_of_sovereign_states').read()

In [ ]:
soup = BeautifulSoup(list_of_flags_page, 'html.parser')

In [ ]:
tables = soup.find_all('table')

In [ ]:
def parse_table(table):

    split_ratio = re.compile('[∶:]').split
    def parse_aspect_ratio(s: str) -> float:
        try:
            n, d = map(int, split_ratio(s))
            return n / d
        except ValueError:
            return float('nan')

    strip_citations = partial(re.compile(r'\s*\[.*\]').sub, "")
    
    table = pd.read_html(StringIO(table.prettify()), flavor='bs4')[0]

    table = table.map(strip_citations)
    table.columns = map(strip_citations, table.columns)
    table.drop('Refs.', axis=1, inplace=True)
    table['Aspect ratio'] = table['Aspect ratio'].apply(parse_aspect_ratio)
    table['Date of latest adoption'] = pd.to_datetime(table['Date of latest adoption'], format='mixed', errors='coerce')
    table['Designer(s)'] = table['Designer(s)'].apply(lambda s: None if s == '—' else s)

    return table

In [ ]:
all_flags = pd.concat(list(map(parse_table, tables)), axis=0, ignore_index=True)
all_flags

In [ ]:
from pathlib import Path

def download_flags(table, folder, idx_offset=0):
    folder  = Path(folder)
    images = table.find_all('img')
    for i, image in enumerate(images):
        src = image['src']
        name = src.split('/')[-1]

        with open(folder / f'{i+idx_offset}-{name}', mode='wb') as f:
            f.write(urllib.request.urlopen('http:' + src).read())

    return len(images) + idx_offset

In [ ]:
last = 0
for table in tables:
    last = download_flags(tables[0], '../data/flags', idx_offset=last)

In [ ]:
# paths = pd.DataFrame.from_records([{'index': int(flag.name.split('-')[0]), 'path': str(flag)} for flag in Path('../data/flags').glob('*')], index='index')
# paths

In [ ]:
all_flags = all_flags.join(paths)
all_flags

In [ ]:
all_flags.to_feather('../data/all_flags.feather')